## Vaccine Distribution using Different Prioritization Factors

In this notebook, we will show how `vaccine_distribution_optimization` function used for determining the optimal allocation of vaccines given the age distribution and case incidences for each location (Province).

Prioritization factors and demographic information that were considered for this study were collected from Open Bangsamoro Database and Philippine Statistics Authority (PSA).

Also, note that the DOH Data Drop Case Information File utilized here came from December 30, 2020. This can be updated with recent data for better recommendation of results.

In [1]:
import os
import sys
import json 

import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.insert(0, os.getcwd() + '/../../')
sys.path.insert(0, os.getcwd() + '/../../abm/')

from abm.utils.modules.optimization.vaccine_distribution_optimization import vaccine_distribution_optimization
from abm.utils.modules import DataManager
from abm.resources.util import DATA_DROP_CSV_FILE, BARMM_DATA_LOC


%matplotlib inline

plt.style.use('ggplot')

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

warnings.simplefilter('ignore')

In [2]:
locations               = ["Basilan", "Lanao del Sur", "Maguindanao", "Sulu", "Tawi-Tawi"]
age_distributions       = [
                            1037200,   # 0 - 9 
                            931800,    # 10 - 19 
                            725000,    # 20 - 29 
                            629100,    # 30 - 39
                            453200,    # 40 - 49
                            296000,    # 50 - 59
                            173700,    # 60 - 69
                            68800,     # 70 - 79
                            15500      # 80 and up
                        ]

population              = [
                            385300,         # Basilan
                            1225700,        # Lanao Del Sur
                            1391100,        # Maguindanao
                            839500,         # Sulu
                            489100          # Tawi-Tawi
                        ]

susceptible_population = []

vaccine_prioritization_weights = {
    "person_health_social_work":        [0.10, 0.32, 0.09, 0.37, 0.11],
    "person_prof_tech":                 [0.06, 0.37, 0.37, 0.12, 0.10],
    "person_admin_support":             [0.01, 0.75, 0.08, 0.10, 0.06],
    "person_education":                 [0.03, 0.77, 0.12, 0.04, 0.05],
    "person_agriculture":               [0.10, 0.75, 0.06, 0.06, 0.03],
    "elderly":                          [0.09, 0.28, 0.32, 0.19, 0.11],
    "social_household_4ps_active":      [0.066, 0.239, 0.306, 0.330, 0.059],
    "social_bayanihan_grant":           [0.08, 0.33, 0.31, 0.17, 0.11],
    "economics_tourists":               [0.1352, 0.1919, 0.3569, 0.0004, 0.3156],
    "economics_marine_fisheries":       [0.03, 0.23, 0.16, 0.13, 0.45],
    "economics_volume_fisheries":       [0.02, 0.02, 0.16, 0.36, 0.44],
    "economics_livestock_inventory":    [0.28, 0.31, 0.24, 0.10, 0.06],
    "economics_volume_corn":            [0.00009, 0.33832, 0.65840, 0.00129, 0.00189]
}

# Assumption here, there would be 5 million vaccine doses available 
vaccines_available      = 5000000   # 2500000 courses = 2 doses per person
data_manager            = DataManager(DATA_DROP_CSV_FILE, BARMM_DATA_LOC, "ProvRes", locations, age_distributions)

In [3]:
# Through DOH Data Drop, case incidence per location were extracted to 
# know what are current number of the susceptible population

location_data = data_manager.get_updated_data("LOCATION_DATA")
for idx, location in enumerate(location_data):
    susceptible_population.append(location["DATA"]["susceptible"] * population[idx])

In [4]:
# Reshape susceptible population
susceptible_population = np.array(susceptible_population).reshape(5, 1)
susceptible_population = susceptible_population.reshape(5, 1)

# Get the percentage of distribution per age groups
age_distributions = np.array(age_distributions) / np.sum(age_distributions)
age_distributions = age_distributions.reshape(9, 1)

# Get the matrix of age group distribution per Province
age_population = susceptible_population * age_distributions.T

susceptible_matrix_population = np.matrix(age_population).T

Here, priority weights were categorized into following groups:

1. Default Priority - includes the following:

    a. Persons with Human Health and Social Work Activities 

    b. Persons with Professional, Scientific and Technical Activities

    c. Persons with Administrative and Support Service Activities

    d. Persons under Education Sectors

    e. Persons under Agriculture, Forestry and Fishing Industry
    
    
2. Social Status- includes the following:

    a. Number of Household Members of Active 4Ps

    b. Bayanihan Grant Allocations
    

3. Economics - includes the following:

    a. Tourist Arrivals

    b. Marine Municipality Fisheries Production

    c. Volume of Fisheries Production

    d. Livestock Inventory

    e. Volume of Corn Production


4. Elderly - percentage distribution of ages 60 and up in each location 


5. All Configurations - includes the above combinations
    

In [5]:
default_priority   = []
social_priority    = []
economic_priority  = []
elderly_priority   = []
all_prioritization = []

default_priority.append(vaccine_prioritization_weights["person_health_social_work"])
default_priority.append(vaccine_prioritization_weights["person_prof_tech"])
default_priority.append(vaccine_prioritization_weights["person_admin_support"])
default_priority.append(vaccine_prioritization_weights["person_education"])
default_priority.append(vaccine_prioritization_weights["person_agriculture"])

social_priority.append(vaccine_prioritization_weights["social_household_4ps_active"])
social_priority.append(vaccine_prioritization_weights["social_bayanihan_grant"])        

economic_priority.append(vaccine_prioritization_weights["economics_tourists"])
economic_priority.append(vaccine_prioritization_weights["economics_marine_fisheries"])
economic_priority.append(vaccine_prioritization_weights["economics_volume_fisheries"])
economic_priority.append(vaccine_prioritization_weights["economics_livestock_inventory"])

elderly_priority.append(vaccine_prioritization_weights["elderly"])

all_prioritization.append(vaccine_prioritization_weights["person_health_social_work"])
all_prioritization.append(vaccine_prioritization_weights["person_prof_tech"])
all_prioritization.append(vaccine_prioritization_weights["person_admin_support"])
all_prioritization.append(vaccine_prioritization_weights["person_education"])
all_prioritization.append(vaccine_prioritization_weights["person_agriculture"])
all_prioritization.append(vaccine_prioritization_weights["social_household_4ps_active"])
all_prioritization.append(vaccine_prioritization_weights["social_bayanihan_grant"])        
all_prioritization.append(vaccine_prioritization_weights["economics_tourists"])
all_prioritization.append(vaccine_prioritization_weights["economics_marine_fisheries"])
all_prioritization.append(vaccine_prioritization_weights["economics_volume_fisheries"])
all_prioritization.append(vaccine_prioritization_weights["economics_livestock_inventory"])
all_prioritization.append(vaccine_prioritization_weights["elderly"])


### Vaccine Distribution without Priority Factors [only referencing on case incidences]

In [6]:
result = vaccine_distribution_optimization(susceptible_matrix_population, vaccines_available,  sub_problems = [])
result_json = json.loads(result)

location_distribution = pd.DataFrame(result_json["optimal_allocation_per_age"])
location_distribution.index = ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80+"]
location_distribution.columns = locations
location_distribution

,Basilan,Lanao del Sur,Maguindanao,Sulu,Tawi-Tawi
0-9,47013.110809,47013.110809,47013.110809,47013.110809,47013.110809
10-19,42235.650455,42235.650455,42235.650455,42235.650455,42235.650455
20-29,32862.037540,32862.037540,32862.037540,32862.037540,32862.037540
30-39,28515.183195,28515.183195,28515.183195,28515.183195,28515.183195
40-49,20542.172984,20542.172984,20542.172984,20542.172984,20542.172984
50-59,13416.776706,13416.776706,13416.776706,13416.776706,13416.776706
60-69,7873.290925,7873.290925,7873.290925,7873.290925,7873.290925
70-79,3118.494045,3118.494045,3118.494045,3118.494045,3118.494045
80+,702.567699,702.567699,702.567699,702.567699,702.567699


### Vaccine Distribution for Prioritizing Frontliners and Essential Workers (Default)

In [7]:
result = vaccine_distribution_optimization(susceptible_matrix_population, vaccines_available,  sub_problems = default_priority)
result_json = json.loads(result)

location_distribution = pd.DataFrame(result_json["optimal_allocation_per_age"])
location_distribution.index = ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80+"]
location_distribution.columns = locations
location_distribution

,Basilan,Lanao del Sur,Maguindanao,Sulu,Tawi-Tawi
0-9,47274.659697,47274.659697,47274.659697,47274.659697,47274.659697
10-19,42470.620811,42470.620811,42470.620811,42470.620811,42470.620811
20-29,33044.859506,33044.859506,33044.859506,33044.859506,33044.859506
30-39,28673.822228,28673.822228,28673.822228,28673.822228,28673.822228
40-49,20656.455625,20656.455625,20656.455625,20656.455625,20656.455625
50-59,13491.418502,13491.418502,13491.418502,13491.418502,13491.418502
60-69,7917.092547,7917.092547,7917.092547,7917.092547,7917.092547
70-79,3135.843219,3135.843219,3135.843219,3135.843219,3135.843219
80+,706.476307,706.476307,706.476307,706.476307,706.476307


### Vaccine Distribution for Prioritizing by Social Status

In [8]:
result = vaccine_distribution_optimization(susceptible_matrix_population, vaccines_available,  sub_problems = social_priority)
result_json = json.loads(result)

location_distribution = pd.DataFrame(result_json["optimal_allocation_per_age"])
location_distribution.index = ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80+"]
location_distribution.columns = locations
location_distribution

,Basilan,Lanao del Sur,Maguindanao,Sulu,Tawi-Tawi
0-9,47158.036000,47158.036000,47158.036000,47158.036000,47158.036000
10-19,42365.848385,42365.848385,42365.848385,42365.848385,42365.848385
20-29,32963.339857,32963.339857,32963.339857,32963.339857,32963.339857
30-39,28603.085661,28603.085661,28603.085661,28603.085661,28603.085661
40-49,20605.497411,20605.497411,20605.497411,20605.497411,20605.497411
50-59,13458.135997,13458.135997,13458.135997,13458.135997,13458.135997
60-69,7897.561563,7897.561563,7897.561563,7897.561563,7897.561563
70-79,3128.107286,3128.107286,3128.107286,3128.107286,3128.107286
80+,704.733473,704.733473,704.733473,704.733473,704.733473


### Vaccine Distribution for Prioritizing by Economic Impact and Revenue Generating Factors

In [9]:
result = vaccine_distribution_optimization(susceptible_matrix_population, vaccines_available,  sub_problems = economic_priority)
result_json = json.loads(result)

location_distribution = pd.DataFrame(result_json["optimal_allocation_per_age"])
location_distribution.index = ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80+"]
location_distribution.columns = locations
location_distribution

,Basilan,Lanao del Sur,Maguindanao,Sulu,Tawi-Tawi
0-9,48391.165929,48391.165929,48391.165929,48391.165929,48391.165929
10-19,43473.667964,43473.667964,43473.667964,43473.667964,43473.667964
20-29,33825.294348,33825.294348,33825.294348,33825.294348,33825.294348
30-39,29351.024379,29351.024379,29351.024379,29351.024379,29351.024379
40-49,21144.308136,21144.308136,21144.308136,21144.308136,21144.308136
50-59,13810.051210,13810.051210,13810.051210,13810.051210,13810.051210
60-69,8104.073970,8104.073970,8104.073970,8104.073970,8104.073970
70-79,3209.903795,3209.903795,3209.903795,3209.903795,3209.903795
80+,723.161465,723.161465,723.161465,723.161465,723.161465


### Vaccine Distribution for Prioritization by All Factors

In [10]:
result = vaccine_distribution_optimization(susceptible_matrix_population, vaccines_available,  sub_problems = all_prioritization)
result_json = json.loads(result)

location_distribution = pd.DataFrame(result_json["optimal_allocation_per_age"])
location_distribution.index = ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80+"]
location_distribution.columns = locations
location_distribution

,Basilan,Lanao del Sur,Maguindanao,Sulu,Tawi-Tawi
0-9,48947.072661,48947.072661,48947.072661,48947.072661,48947.072661
10-19,43973.083596,43973.083596,43973.083596,43973.083596,43973.083596
20-29,34213.871654,34213.871654,34213.871654,34213.871654,34213.871654
30-39,29688.202286,29688.202286,29688.202286,29688.202286,29688.202286
40-49,21387.209150,21387.209150,21387.209150,21387.209150,21387.209150
50-59,13968.697944,13968.697944,13968.697944,13968.697944,13968.697944
60-69,8197.171733,8197.171733,8197.171733,8197.171733,8197.171733
70-79,3246.778441,3246.778441,3246.778441,3246.778441,3246.778441
80+,731.468980,731.468980,731.468980,731.468980,731.468980
